# Food Items

In [ ]:
# .venv\Scripts\activate

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import time

In [2]:
url = "https://i.fnri.dost.gov.ph/fct/library/food_content/all"
driver_path = "../driver/chromedriver.exe"
export_path = "../ph_food_item_options_raw_foodlist.csv"

In [3]:
serv = Service(driver_path)
driver = Chrome(service=serv)

driver.get(url)

In [11]:
# time.sleep(40)

In [5]:
# nutr_info = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div')
# nutr_info_each = nutr_info.find_elements(By.TAG_NAME, 'div')
# nutr_info_each


In [71]:
# driver.find_element(By.ID, 'A001_data')

''

In [4]:
records_per_page_btn = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div/div[1543]/div[2]/div[3]/div[1]/span[2]/span/button')
records_per_page_btn.click()


In [5]:
records_per_page_100 = driver.find_element(By.XPATH, "/html/body/div[2]/div[3]/div/div/div[1543]/div[2]/div[3]/div[1]/span[2]/span/ul/li[4]/a")
records_per_page_100.click()

In [ ]:
records_per_page_dropdown = driver.find_element(By.CLASS_NAME, 'dropdown-menu')
records_per_page_list = records_per_page_dropdown.find_elements(By.TAG_NAME, 'li')
# records_per_page_dropdown[-1].text

In [11]:
ended = False
try: 
    next_page_btn = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div/div[1543]/div[2]/div[3]/div[2]/ul/li[8]/a')
    next_page_btn.click() 
except:
    ended = True
print(ended)


True


In [10]:
food_datatable = driver.find_element(By.XPATH, "/html/body/div[2]/div[3]/div/div/div[1543]/div[2]/div[2]/table/tbody")
rows = food_datatable.find_elements(By.TAG_NAME, 'tr')

In [14]:
row = rows[2]

vals = []
cols = row.find_elements(By.TAG_NAME, 'td')
for col in cols[:-1]:
    vals.append(col.text)

food_id = vals[0]
vals

['R102', 'Scrambled egg, prep, w/ MLP', 'N/A', 'N/A', '100%']

In [57]:
cols[-1].find_elements(By.TAG_NAME, 'button')[0].click()

In [ ]:
nutr_popout = driver.find_element(By.XPATH, '//*[@id="R102_data"]/div/div/div/div[1]')

In [60]:
nutr_header = driver.find_element(By.XPATH, '//*[@id="' + food_id + '_data"]/div/div/div/ul')
nutr_sections = nutr_header.find_elements(By.TAG_NAME, 'li')

In [64]:
nutr_basis = driver.find_element(By.XPATH, '//*[@id="proximates' + food_id + '"]/ul/li[1]/span').text
nutr_basis

'Amount per 100 g E.P.'

In [ ]:
nutr_datatable = driver.find_element(By.XPATH, '//*[@id="proximates' + food_id + '"]/ul')
nutr_datatable

In [63]:
section = nutr_sections[1]
section.click()

In [56]:

close_btn = driver.find_element(By.XPATH, '//*[@id="' + food_id + '_data"]/div/div/div/div[2]/button')
close_btn.click()

In [15]:
driver.quit()


# Food Nutr

In [1]:
import pandas as pd

df_foodlist = pd.read_cs
v('../ph_food_item_options_raw_foodlist.csv', delimiter='|')
df_foodlist

,food_id,food_name,scientific_name,common_name,edible_portion
0,A001,"Corn grits, white",Zea mays,"Mais, durog, puti",100%
1,A002,"Corn grits, yellow",Zea mays,"Mais, durog, dilaw",100%
2,A003,"Corn on cob, white",Zea mays,"Mais sa busal, puti",30%
3,A004,"Corn on cob, yellow",Zea mays,"Mais sa busal, dilaw",39%
4,A005,"Corn on cob, yellow, boiled",Zea mays,"Mais sa busal, dilaw, nilaga",54%
...,...,...,...,...,...
1537,D304,Ginger,Zingiber officinale,Luya,74%
1538,D305,Turmeric,Curcuma longa L.,Luyang dilaw,71%
1539,H023,"Egg mass, sea hare",Dolabella auricularia,Lukot,100%
1540,Q062,"Tea, prep w/ water, no sugar",NaN,NaN,100%


In [2]:
foodlist = df_foodlist['food_id'].values
foodlist

array(['A001', 'A002', 'A003', ..., 'H023', 'Q062', 'T012'], dtype=object)

In [3]:
import lxml.html
import urllib

dom_str = urllib.request.urlopen('https://i.fnri.dost.gov.ph/fct/library/food_content/all').read()
dom_lxml = lxml.html.fromstring(dom_str)

# description = lxml_mysite.xpath("//meta[@name='description']")[0] # meta tag description
# text = description.get('content') # content attribute of the tag

export_path = '../ph_food_item_options_raw_foodnutr.csv'

In [4]:
from tqdm import tqdm
for food_id in tqdm(foodlist):
    nutr_sections = dom_lxml.xpath('//div[@id="' + food_id + '_data"]/div/div/div/div[1]')[0]

    nutr_info = []

    nutr_base = dom_lxml.xpath('//div[@id="proximates' + food_id + '"]/ul/li[1]/span')[0].text
    nutr_info.append([food_id, 'base', nutr_base])

    for section in nutr_sections.xpath('./div'):
        # print(section.get('id'))
        nutr_table = section.xpath('./ul/li')
        # print(len(nutr_table))
        nutr_base = nutr_table[0].xpath('./span')[0].text
        # print(nutr_table[0].xpath('./span')[0].text)

        for row in nutr_table[1:]:
            cols = row.xpath('./div')
            if cols == []:
                continue
            nutr_info.append([food_id, cols[0].text_content(), cols[1].text_content()])

    with open(export_path, "a", encoding="utf-8") as f:
        for row in nutr_info:
            f.write("|".join(row) + '\n')



100%|██████████| 1542/1542 [07:49<00:00,  3.28it/s]


In [ ]:
#### test 

In [25]:
food_id = foodlist[0]
nutr_sections = dom_lxml.xpath('//div[@id="' + food_id + '_data"]/div/div/div/div[1]')[0]

In [51]:
nutr_info = []

nutr_base = dom_lxml.xpath('//div[@id="proximates' + food_id + '"]/ul/li[1]/span')[0].text
nutr_info.append(['base', nutr_base])

for section in nutr_sections.xpath('./div'):
    # print(section.get('id'))
    nutr_table = section.xpath('./ul/li')
    # print(len(nutr_table))
    nutr_base = nutr_table[0].xpath('./span')[0].text
    # print(nutr_table[0].xpath('./span')[0].text)

    for row in nutr_table[1:]:
        cols = row.xpath('./div')
        if cols == []:
            continue
        nutr_info.append([cols[0].text_content(), cols[1].text_content()])

nutr_info

[['base', 'Amount per 100 g E.P.'],
 ['Water (g)', '12.2'],
 ['Energy, calculated (kcal)', '357'],
 ['Protein (g)', '8.3'],
 ['Total Fat (g)', '1.5'],
 ['Carbohydrate, total (g)', '77.5'],
 ['Ash, total (g)', '0.5'],
 ['Fiber, total dietary (g)', '4.5'],
 ['Sugars, total (g)', '0.6'],
 ['Calcium, Ca (mg)', '11'],
 ['Phosphorus, P (mg)', '86'],
 ['Iron, Fe (mg)', '0.4'],
 ['Sodium, Na (mg)', '1'],
 ['Retinol, Vitamin A (µg)', '0'],
 ['beta-Carotene (µg)', '0'],
 ['Retinol Activity Equivalent, RAE (µg)', '0'],
 ['Thiamin, Vitamin B1 (mg)', '0.08'],
 ['Riboflavin, Vitamin B2 (mg)', '0.05'],
 ['Niacin (mg)', '0.6'],
 ['Ascorbic Acid, Vitamin C (mg)', '0'],
 ['Fatty acids, saturated, total (g)', '0.29'],
 ['Fatty acids, monounsaturated, total (g)', '0.30'],
 ['Fatty acids, polyunsaturated, total(g)', '0.75'],
 ['Cholesterol (mg)', '0']]